In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager

import time

from bs4 import BeautifulSoup

In [2]:
# this will download webdriver and stored in cache
# if it already in cache - then it picks from cache
# and then it open web-browser
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_linux64.zip
[WDM] - Driver has been saved in cache [/home/sumanshu/.wdm/drivers/chromedriver/linux64/87.0.4280.88]


In [3]:
# this will direct to the mentioned URL - in already open chrome browser
driver.get("https://www.youtube.com/user/jacksfilms/videos")

In [4]:
# There are so many videos
# as we scroll down - the other videos on same page get loaded
# so we need to tell chrome to press END button - so that all videos get loaded on same page
# there are total 1246 videos right now and on 1 page around 15-20 videos
# so we need to press END key approx 62 times (1246/20)
# after pressing one END key - we will wait for 3-4 sec to get page loaded fully

for _ in range(60):
    driver.find_element_by_tag_name("body").send_keys(Keys.END)
    time.sleep(4)

In [5]:
# All the HTML of entire page come into below variable
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
videos = soup.find_all('div', {'id':'dismissable'})
master_list = []
for video in videos:
    data_dict = {}
    data_dict['title'] = video.find('a', {'id':'video-title'}).text
    data_dict['video_url'] = 'https://www.youtube.com/' + video.find('a', {'id':'video-title'})['href']
    meta = video.find('div', {'id':'metadata-line'}).find_all('span')
    data_dict['views'] = meta[0].text
    data_dict['video_age'] = meta[1].text
    master_list.append(data_dict)

import pandas as pd
youtube_df = pd.DataFrame(master_list)
youtube_df.head(6)

,title,video_url,views,video_age
0,Royalty Free Christmas Songs 7,https://www.youtube.com//watch?v=C8OJMHQ1ZZs,355K views,1 day ago
1,FIX YOUR PIX 4,https://www.youtube.com//watch?v=MoxnDUxc1a8,532K views,1 week ago
2,WHO IS THE FUNNIEST YOUTUBER?!,https://www.youtube.com//watch?v=VOK0X7K4RZY,515K views,1 week ago
3,My wife is better at this game than me,https://www.youtube.com//watch?v=lL-EKMziUas,318K views,2 weeks ago
4,We wrote an original Hallmark Christmas movie ...,https://www.youtube.com//watch?v=zOQXKVGvCXw,430K views,2 weeks ago
5,"The terrifying, confusing world of ""Princess C...",https://www.youtube.com//watch?v=9EK8OE8Svm4,481K views,3 weeks ago


In [6]:
def convert_views(df):
    if 'K' in df['views']:
        views = float(df['views'].split('K')[0])*1000
        return views
    elif 'M' in df['views']:
        views = float(df['views'].split('M')[0])*1000000
        return views   

In [7]:
youtube_df['Clean-Views'] = youtube_df.apply(convert_views, axis=1)
youtube_df['Clean-Views'] = youtube_df['Clean-Views'].astype(int)
youtube_df.head(6)

,title,video_url,views,video_age,Clean-Views
0,Royalty Free Christmas Songs 7,https://www.youtube.com//watch?v=C8OJMHQ1ZZs,355K views,1 day ago,355000
1,FIX YOUR PIX 4,https://www.youtube.com//watch?v=MoxnDUxc1a8,532K views,1 week ago,532000
2,WHO IS THE FUNNIEST YOUTUBER?!,https://www.youtube.com//watch?v=VOK0X7K4RZY,515K views,1 week ago,515000
3,My wife is better at this game than me,https://www.youtube.com//watch?v=lL-EKMziUas,318K views,2 weeks ago,318000
4,We wrote an original Hallmark Christmas movie ...,https://www.youtube.com//watch?v=zOQXKVGvCXw,430K views,2 weeks ago,430000
5,"The terrifying, confusing world of ""Princess C...",https://www.youtube.com//watch?v=9EK8OE8Svm4,481K views,3 weeks ago,481000


In [9]:
driver = webdriver.Chrome(ChromeDriverManager().install())
import time

master_list = []
for url in youtube_df['video_url']:
    dic = {}
    driver.get(url)
    time.sleep(5)
    driver.find_element_by_tag_name("body")
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    video_views = int(soup.find('span', {'class':"view-count"}).text.replace('views','').replace(',',''))
    video_date = soup.find('div',{'id':"date"})
    video_date = video_date.text.replace('•','')
    dic['url'] = url
    dic['video_views'] = video_views
    dic['video_date'] = video_date
    master_list.append(dic)
    break

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/home/sumanshu/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache


In [ ]:
youtube_df1 = pd.DataFrame(master_list)
youtube_df1.head()

In [ ]:
final_df = pd.concat([youtube_df,youtube_df1], axis=1)

In [ ]:
final_df.to_csv('jackfilms.csv', index=False)